# This notebook works on extracting unique named entities and organizations from KDD papers and passing them lists.  

In [198]:
import pandas as pd
import numpy as np
import nltk
import os
import subprocess
import unicodedata
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk import Tree
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import re
from operator import itemgetter
import polyglot
import string
from emailextractor import file_to_str, get_emails # paste code to .py file from following link and save within your environment path to call it: https://gist.github.com/dideler/5219706

'''
Severl issues extended the time to complete this work
that were not related to the coding itself.  I had two
Anaconda distros installed on my computer.  This led
to problems with importing modules because the paths
for installation and retrieval were mixed.  I had to
uninstall both anacondas, reinstall, and then recreate
my virtual environment.  Imports and installs worked cleanly

To create ipython shells in a virtual environment, use:
http://stackoverflow.com/questions/30492623/using-both-python-2-x-and-python-3-x-in-ipython-notebook

''';

In [199]:
path        = os.path.abspath(os.getcwd())
TESTDIR     = os.path.normpath(os.path.join(os.path.expanduser("~"),"projects","LC3-Creations", "examples","KDDsample"))


In [200]:
'''
I experienced unicode problems early on.  Everytime I had an error, I scoured the internet for solutions. Here's the credit.



- For Typeerror codes using subprocess to convert pdf2txt output to straight unicode --> http://stackoverflow.com/questions/33283603/python-popen-communicate-str-encodeencoding-utf-8-errors-ignore-cr
- For problems with ASCII characters --> http://stackoverflow.com/questions/175240/how-do-i-convert-a-files-format-from-unicode-to-ascii-using-python
- For unicode characters left in unicode converted to a string  --> http://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces-using-python

'''


a = unicode(subprocess.check_output(['pdf2txt.py',str(os.path.normpath(os.path.join(TESTDIR,"p1.pdf")))]),errors='ignore')
document = filter(lambda x: x in string.printable,unicodedata.normalize('NFKD', a).encode('ascii','ignore').decode('unicode_escape').encode('ascii','ignore'))
lower = document.lower()

In [201]:
print document

Online Controlled Experiments: 

Lessons from Running A/B/n Tests for 12 Years 

 

Ron Kohavi 

Distinguished Engineer & General Manager,  

Analysis & Experimentation 

Microsoft 

Bellevue, WA, USA 

RonKohavi@outlook.com  

in 

He 

2005 

 
BIO 
Ronny  Kohavi is  a  Microsoft 
Distinguished  Engineer  and 
for 
the  General  Manager 
and 
Microsofts  Analysis 
at 
Experimentation 
team 
Microsoft. 
joined 
Microsoft 
and 
founded  the  Experimentation 
Platform  team  in  2006.  He 
was previously the director of 
data  mining  and  personalization  at  Amazon.com,  and  the  Vice 
President  of  Business  Intelligence  at  Blue  Martini  Software, 
which  went  public  in  2000,  and  later  acquired  by  Red  Prairie. 
Prior to joining Blue Martini, Kohavi managed MineSet project, 
Silicon  Graphics  award-winning  product  for  data  mining  and 
visualization. He joined Silicon Graphics after getting a Ph.D. in 
Machine  Learning  from  Stanford  University,  where  he  led  

In [202]:
"returns named entity chunks in a given text"
tagged = nltk.pos_tag(nltk.word_tokenize(re.sub('[\s]'," ", document)))
entities = nltk.chunk.ne_chunk(tagged)

In [203]:
# putting it in similar formats for visuals
for l in entities:
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'PERSON':
            print " ".join(map(itemgetter(0), l))+","+l.label()
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'ORGANIZATION':
            print " ".join(map(itemgetter(0), l))+","+l.label()
                

Online,PERSON
Ron Kohavi Distinguished Engineer,PERSON
General Manager,ORGANIZATION
Analysis,PERSON
Experimentation Microsoft Bellevue,ORGANIZATION
WA,ORGANIZATION
USA,ORGANIZATION
BIO Ronny Kohavi,ORGANIZATION
Microsoft Distinguished Engineer,ORGANIZATION
General Manager,ORGANIZATION
Microsofts Analysis,PERSON
Experimentation,ORGANIZATION
Microsoft,PERSON
Microsoft,PERSON
Experimentation Platform,ORGANIZATION
Amazon.com,ORGANIZATION
Business Intelligence,ORGANIZATION
Red Prairie,PERSON
Blue Martini,PERSON
Kohavi,PERSON
MineSet,ORGANIZATION
Silicon Graphics,PERSON
Silicon Graphics,PERSON
Stanford University,ORGANIZATION
Machine,ORGANIZATION
MineSet,ORGANIZATION
Kohavi,PERSON
Technion,ORGANIZATION
General,ORGANIZATION
KDD,ORGANIZATION
KDD,ORGANIZATION
Jim Gray,PERSON
KDD,ORGANIZATION
Carla Brodley,PERSON
National Academy,ORGANIZATION
PAKDD,ORGANIZATION
KDD,ORGANIZATION
EC,ORGANIZATION
Recsys,ORGANIZATION
Computer Science,ORGANIZATION
Microsofts Bing,PERSON
eBay,ORGANIZATION
Netflix,PERS

Below, I established two lists to hold the values that I extract from the text.  This itemgetter function will check for unique values.  First, I iterate over the extracted entities and see if the objects is a nltk.tree.Tree with a "Person" label.  If it is, and the length is equal to 1 (first or last name only), I append that value to the list. If it's larger, I iterate of the entity tree and pull out the first value only using itemgetter.  Then, I join the values from the list and append it to the destination list.  

# Getting a list out of NLTK's standard NE chunker

In [204]:
from operator import itemgetter

persons = []
organizations = []
locations =[]

for l in entities:
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'PERSON':
            if len(l)== 1:
                if l[0][0] in persons:
                    pass
                else:
                    persons.append(l[0][0])
            else:
                if " ".join(map(itemgetter(0), l)) in persons:
                    pass
                else:
                    persons.append(" ".join(map(itemgetter(0), l)))
                    
for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'ORGANIZATION':
            if len(o)== 1:
                if o[0][0] in organizations:
                    pass
                else:
                    organizations.append(o[0][0])
            else:
                if " ".join(map(itemgetter(0), o)) in organizations:
                    pass
                else:
                    organizations.append(" ".join(map(itemgetter(0), o)))
                    
for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'LOCATION':
            if len(o)== 1:
                if o[0][0] in locations:
                    pass
                else:
                    locations.append(o[0][0])
            else:
                if " ".join(map(itemgetter(0), o)) in locations:
                    pass
                else:
                    locations.append(" ".join(map(itemgetter(0), o)))
                    
                
print persons
print
print
print organizations
print
print
print locations

['Online', 'Ron Kohavi Distinguished Engineer', 'Analysis', 'Microsofts Analysis', 'Microsoft', 'Red Prairie', 'Blue Martini', 'Kohavi', 'Silicon Graphics', 'Jim Gray', 'Carla Brodley', 'Microsofts Bing', 'Netflix', 'Twitter', 'Sir Ronald', 'Subject Descriptors', 'Terms Measurement', 'Copyright']


['General Manager', 'Experimentation Microsoft Bellevue', 'WA', 'USA', 'BIO Ronny Kohavi', 'Microsoft Distinguished Engineer', 'Experimentation', 'Experimentation Platform', 'Amazon.com', 'Business Intelligence', 'MineSet', 'Stanford University', 'Machine', 'Technion', 'General', 'KDD', 'National Academy', 'PAKDD', 'EC', 'Recsys', 'Computer Science', 'eBay', 'LinkedIn', 'Rothamsted', 'Bing', 'Experimentation Keywords Controlled', 'NSW', 'ACM']


[]


In [205]:
from operator import itemgetter

for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'ORGANIZATION' or o.label() == 'GPE':
            if len(o)>1:
                print " ".join(map(itemgetter(0), o))

General Manager
Experimentation Microsoft Bellevue
BIO Ronny Kohavi
Microsoft Distinguished Engineer
General Manager
Experimentation Platform
Business Intelligence
Stanford University
National Academy
Computer Science
Experimentation Keywords Controlled


I tried to iterate over the extracted list of entities to get a better break between person's and their university name.  

In [206]:
tokens = [nltk.word_tokenize(l) for l in persons]
fin = [nltk.chunk.ne_chunk(nltk.pos_tag(l)) for l in tokens]
fin;

# Creating lists of named entities from Stanford's NER model

This function looks though an extracted stanford ner list, and finds continuous entitiy labels.  This should create first name, last name records of entities.  

In [207]:
st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz',
       '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
       encoding='utf-8')
tokenized_text = word_tokenize(re.sub('[\s]'," ", document))
stanentities = st.tag(tokenized_text)


def get_continuous_chunks(tagged_sent):
    continuous_chunk = []
    current_chunk = []

    for token, tag in tagged_sent:
        if tag != "O":
            current_chunk.append((token, tag))
        else:
            if current_chunk: # if the current chunk is not empty
                continuous_chunk.append(current_chunk)
                current_chunk = []
    # Flush the final current_chunk into the continuous_chunk, if any.
    if current_chunk:
        continuous_chunk.append(current_chunk)
    return continuous_chunk

ne_tagged_sent = [('Rami', 'PERSON'), ('Eid', 'PERSON'), ('is', 'O'), ('studying', 'O'), ('at', 'O'), ('Stony', 'ORGANIZATION'), ('Brook', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('in', 'O'), ('NY', 'LOCATION')]

named_entities = get_continuous_chunks(stanentities)
named_entities_str = [" ".join([token for token, tag in ne]) for ne in named_entities]
named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]


In [208]:
named_entities = get_continuous_chunks(stanentities)
named_entities_str = [" ".join([token for token, tag in ne]) for ne in named_entities]
named_entities_str_tag

[(u'Years Ron Kohavi Distinguished Engineer & General Manager , Analysis & Experimentation Microsoft Bellevue',
  u'ORGANIZATION'),
 (u'USA', u'ORGANIZATION'),
 (u'Ronny Kohavi', u'PERSON'),
 (u'Microsoft', u'ORGANIZATION'),
 (u'General Manager', u'ORGANIZATION'),
 (u'Experimentation', u'ORGANIZATION'),
 (u'Microsoft', u'ORGANIZATION'),
 (u'Microsoft', u'ORGANIZATION'),
 (u'2006', u'DATE'),
 (u'Amazon.com', u'ORGANIZATION'),
 (u'Blue Martini Software', u'ORGANIZATION'),
 (u'2000', u'DATE'),
 (u'Red Prairie', u'ORGANIZATION'),
 (u'Blue Martini', u'ORGANIZATION'),
 (u'Kohavi', u'PERSON'),
 (u'Silicon Graphics', u'ORGANIZATION'),
 (u'Machine Learning', u'ORGANIZATION'),
 (u'Stanford University', u'ORGANIZATION'),
 (u'Machine Learning', u'ORGANIZATION'),
 (u'Blue Martini Software', u'ORGANIZATION'),
 (u'BA', u'ORGANIZATION'),
 (u'Technion', u'LOCATION'),
 (u'Israel', u'LOCATION'),
 (u'KDD 2004', u'ORGANIZATION'),
 (u'KDD', u'ORGANIZATION'),
 (u'Jim Gray', u'PERSON'),
 (u'KDD', u'ORGANIZATI

In [209]:
new =[word_tokenize(l) for l in persons]
stan = [st.tag(l) for l in new]
stan;

In [210]:
compare=[]
for l,m in named_entities_str_tag:
    l=re.sub('(\])'," ",l).strip()
    if m == 'PERSON':
        if l in compare:
            pass
        else:
            compare.append(l)
    else:
        pass
compare

[u'Ronny Kohavi',
 u'Kohavi',
 u'Jim Gray',
 u'Carla Brodley',
 u'Microsofts Bing',
 u'Ronald A. Fishers']

In [211]:
list1 = range(10)

In [212]:
list2 = [i for i in xrange(7,17,1)]


In [213]:
set(list1) & set(list2)

{7, 8, 9}

In [214]:
def parts_of_speech(corpus):
    "returns named entity chunks in a given text"
    tagged = nltk.pos_tag(nltk.word_tokenize(corpus))
    entities = nltk.chunk.ne_chunk(tagged)
    # Another entity extractor
    st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz',
           '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
           encoding='utf-8')
    tokenized_text = word_tokenize(corpus)
    stanentities = st.tag(tokenized_text)
    return entities
def find_entities(chunks):
    "given list of tagged parts of speech, returns unique named entities"

    def traverse(tree):
        "recursively traverses an nltk.tree.Tree to find named entities"
        entity_names = []
    
        if hasattr(tree, 'node') and tree.node:
            if tree.node == 'NE':
                entity_names.append(' '.join([child[0] for child in tree]))
            else:
                for child in tree:
                    entity_names.extend(traverse(child))
    
        return entity_names
    
    named_entities = []
    
    for chunk in chunks:
        entities = sorted(list(set([word for tree in chunk
                            for word in traverse(tree)])))
        for e in entities:
            if e not in named_entities:
                named_entities.append(e)
    return named_entities

In [215]:
named_entities

[[(u'Years', u'ORGANIZATION'),
  (u'Ron', u'ORGANIZATION'),
  (u'Kohavi', u'ORGANIZATION'),
  (u'Distinguished', u'ORGANIZATION'),
  (u'Engineer', u'ORGANIZATION'),
  (u'&', u'ORGANIZATION'),
  (u'General', u'ORGANIZATION'),
  (u'Manager', u'ORGANIZATION'),
  (u',', u'ORGANIZATION'),
  (u'Analysis', u'ORGANIZATION'),
  (u'&', u'ORGANIZATION'),
  (u'Experimentation', u'ORGANIZATION'),
  (u'Microsoft', u'ORGANIZATION'),
  (u'Bellevue', u'ORGANIZATION')],
 [(u'USA', u'ORGANIZATION')],
 [(u'Ronny', u'PERSON'), (u'Kohavi', u'PERSON')],
 [(u'Microsoft', u'ORGANIZATION')],
 [(u'General', u'ORGANIZATION'), (u'Manager', u'ORGANIZATION')],
 [(u'Experimentation', u'ORGANIZATION')],
 [(u'Microsoft', u'ORGANIZATION')],
 [(u'Microsoft', u'ORGANIZATION')],
 [(u'2006', u'DATE')],
 [(u'Amazon.com', u'ORGANIZATION')],
 [(u'Blue', u'ORGANIZATION'),
  (u'Martini', u'ORGANIZATION'),
  (u'Software', u'ORGANIZATION')],
 [(u'2000', u'DATE')],
 [(u'Red', u'ORGANIZATION'), (u'Prairie', u'ORGANIZATION')],
 [(u'B

# Extracting entities and creating lists using Polyglot

In [216]:
from polyglot.text import Text
e=Text(re.sub('[\s]'," ",document[:10000])).entities

This code iterates over the polyglot extracted entities and creates a list of person, locations, and organizations

In [217]:
import itertools
import unicodedata

def extraction(corpus):
    
    # extract entities from a single string; remove whitespace characters
    try:
        e = Text(re.sub('[\s]'," ",corpus)).entities
    except:
        pass #e = Text(re.sub("(r'(x0)'," ","(re.sub('[\s]'," ",corpus)))).entities
    
    current_person =[]
    persons =[]
    current_org=[]
    organizations=[]
    current_loc=[]
    locations=[]

    for l in e:
        if l.tag == 'I-PER':
            for m in l:
                current_person.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_person: # if the current chunk is not empty
                        persons.append(" ".join(current_person))
                        current_person = []
        elif l.tag == 'I-ORG':
            for m in l:
                current_org.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_org: # if the current chunk is not empty
                        organizations.append(" ".join(current_org))
                        current_org = []
        elif l.tag == 'I-LOC':
            for m in l:
                current_loc.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_loc: # if the current chunk is not empty
                        locations.append(" ".join(current_loc))
                        current_loc = []
    results = {}
    results['persons']=persons
    results['organizations']=organizations
    results['locations']=locations
    
    return results

In [218]:
extraction(document)['persons']

['Ron Kohavi',
 'Ronny Kohavi',
 'Blue Martini',
 'Kohavi',
 'Kohavi',
 'Jim Gray',
 'Carla Brodley',
 'Microsofts Bing',
 'Sir Ronald A',
 'KDD15']

In [221]:
document;

# Truth Sets to test extraction accuracy

In [222]:
#p19.pdf

p19pdf_authors=['Tim Althoff*','Xin Luna Dong','Kevin Murphy','Safa Alai','Van Dang','Wei Zhang']
p19pdf_author_organizations=['Computer Science Department','Stanford University','Google']
p19pdf_author_locations=['Stanford, CA','Stanford','CA','Google','1600 Amphitheatre Parkway, Mountain View, CA 94043','1600 Amphitheatre Parkway','Mountain View']

p19pdf_references_authors =['A. Ahmed', 'C. H. Teo', 'S. Vishwanathan','A. Smola','J. Allan', 'R. Gupta', 'V. Khandelwal',
                           'D. Graus', 'M.-H. Peetz', 'D. Odijk', 'O. de Rooij', 'M. de Rijke','T. Huet', 'J. Biega', 
                            'F. M. Suchanek','H. Ji', 'T. Cassidy', 'Q. Li','S. Tamang', 'A. Kannan', 'S. Baker', 'K. Ramnath', 
                            'J. Fiss', 'D. Lin', 'L. Vanderwende',  'R. Ansary', 'A. Kapoor', 'Q. Ke', 'M. Uyttendaele',
                           'S. M. Katz','A. Krause','D. Golovin','J. Leskovec', 'A. Krause', 'C. Guestrin', 'C. Faloutsos', 
                            'J. VanBriesen','N. Glance','J. Li','C. Cardie','J. Li','C. Cardie','C.-Y. Lin','H. Lin','J. A. Bilmes'
                           'X. Ling','D. S. Weld', 'A. Mazeika', 'T. Tylenda','G. Weikum','M. Minoux', 'G. L. Nemhauser', 'L. A. Wolsey',
                            'M. L. Fisher','R. Qian','D. Shahaf', 'C. Guestrin','E. Horvitz','T. Althoff', 'X. L. Dong', 'K. Murphy', 'S. Alai',
                            'V. Dang','W. Zhang','R. A. Baeza-Yates', 'B. Ribeiro-Neto', 'D. Shahaf', 'J. Yang', 'C. Suen', 'J. Jacobs', 'H. Wang', 'J. Leskovec',
                           'W. Shen', 'J. Wang', 'J. Han','D. Bamman', 'N. Smith','K. Bollacker', 'C. Evans', 'P. Paritosh', 'T. Sturge', 'J. Taylor',
                           'R. Sipos', 'A. Swaminathan', 'P. Shivaswamy', 'T. Joachims','K. Sprck Jones','G. Calinescu', 'C. Chekuri', 'M. Pl','J. Vondrk',
                           'F. M. Suchanek', 'G. Kasneci','G. Weikum', 'J. Carbonell' ,'J. Goldstein','B. Carterette', 'P. N. Bennett', 'D. M. Chickering',
                            'S. T. Dumais','A. Dasgupta', 'R. Kumar','S. Ravi','Q. X. Do', 'W. Lu', 'D. Roth','X. Dong', 'E. Gabrilovich', 'G. Heitz', 'W. Horn', 
                            'N. Lao', 'K. Murphy',  'T. Strohmann', 'S. Sun','W. Zhang', 'M. Dubinko', 'R. Kumar', 'J. Magnani', 'J. Novak', 'P. Raghavan','A. Tomkins',
                           'U. Feige','F. M. Suchanek','N. Preda','R. Swan','J. Allan', 'T. Tran', 'A. Ceroni', 'M. Georgescu', 'K. D. Naini', 'M. Fisichella',
                           'T. A. Tuan', 'S. Elbassuoni', 'N. Preda','G. Weikum','Y. Wang', 'M. Zhu', 'L. Qu', 'M. Spaniol', 'G. Weikum',
                           'G. Weikum', 'N. Ntarmos', 'M. Spaniol', 'P. Triantallou', 'A. A. Benczr',  'S. Kirkpatrick', 'P. Rigaux','M. Williamson',
                           'X. W. Zhao', 'Y. Guo', 'R. Yan', 'Y. He','X. Li']
p19pdf_allauthors=['Tim Althoff*','Xin Luna Dong','Kevin Murphy','Safa Alai','Van Dang','Wei Zhang','A. Ahmed', 'C. H. Teo', 'S. Vishwanathan','A. Smola','J. Allan', 'R. Gupta', 'V. Khandelwal',
                           'D. Graus', 'M.-H. Peetz', 'D. Odijk', 'O. de Rooij', 'M. de Rijke','T. Huet', 'J. Biega', 
                            'F. M. Suchanek','H. Ji', 'T. Cassidy', 'Q. Li','S. Tamang', 'A. Kannan', 'S. Baker', 'K. Ramnath', 
                            'J. Fiss', 'D. Lin', 'L. Vanderwende',  'R. Ansary', 'A. Kapoor', 'Q. Ke', 'M. Uyttendaele',
                           'S. M. Katz','A. Krause','D. Golovin','J. Leskovec', 'A. Krause', 'C. Guestrin', 'C. Faloutsos', 
                            'J. VanBriesen','N. Glance','J. Li','C. Cardie','J. Li','C. Cardie','C.-Y. Lin','H. Lin','J. A. Bilmes'
                           'X. Ling','D. S. Weld', 'A. Mazeika', 'T. Tylenda','G. Weikum','M. Minoux', 'G. L. Nemhauser', 'L. A. Wolsey',
                            'M. L. Fisher','R. Qian','D. Shahaf', 'C. Guestrin','E. Horvitz','T. Althoff', 'X. L. Dong', 'K. Murphy', 'S. Alai',
                            'V. Dang','W. Zhang','R. A. Baeza-Yates', 'B. Ribeiro-Neto', 'D. Shahaf', 'J. Yang', 'C. Suen', 'J. Jacobs', 'H. Wang', 'J. Leskovec',
                           'W. Shen', 'J. Wang', 'J. Han','D. Bamman', 'N. Smith','K. Bollacker', 'C. Evans', 'P. Paritosh', 'T. Sturge', 'J. Taylor',
                           'R. Sipos', 'A. Swaminathan', 'P. Shivaswamy', 'T. Joachims','K. Sprck Jones','G. Calinescu', 'C. Chekuri', 'M. Pl','J. Vondrk',
                           'F. M. Suchanek', 'G. Kasneci','G. Weikum', 'J. Carbonell' ,'J. Goldstein','B. Carterette', 'P. N. Bennett', 'D. M. Chickering',
                            'S. T. Dumais','A. Dasgupta', 'R. Kumar','S. Ravi','Q. X. Do', 'W. Lu', 'D. Roth','X. Dong', 'E. Gabrilovich', 'G. Heitz', 'W. Horn', 
                            'N. Lao', 'K. Murphy',  'T. Strohmann', 'S. Sun','W. Zhang', 'M. Dubinko', 'R. Kumar', 'J. Magnani', 'J. Novak', 'P. Raghavan','A. Tomkins',
                           'U. Feige','F. M. Suchanek','N. Preda','R. Swan','J. Allan', 'T. Tran', 'A. Ceroni', 'M. Georgescu', 'K. D. Naini', 'M. Fisichella',
                           'T. A. Tuan', 'S. Elbassuoni', 'N. Preda','G. Weikum','Y. Wang', 'M. Zhu', 'L. Qu', 'M. Spaniol', 'G. Weikum',
                           'G. Weikum', 'N. Ntarmos', 'M. Spaniol', 'P. Triantallou', 'A. A. Benczr',  'S. Kirkpatrick', 'P. Rigaux','M. Williamson',
                           'X. W. Zhao', 'Y. Guo', 'R. Yan', 'Y. He','X. Li']

print "There are %r authors" % len(p19pdf_authors)
print  # white space
print "There are %r author organizations" %len(p19pdf_author_organizations)
print 
print "There are %r author locations" % len(p19pdf_author_locations)
print  
print "There are %r authors in the references" %len(p19pdf_references_authors)

There are 6 authors

There are 3 author organizations

There are 7 author locations

There are 152 authors in the references


In [223]:
#p29.pdf

p29pdf_authors=['Laurent Amsaleg','Stéphane Girard','Oussama Chelly','Teddy Furon','Michael E. Houle','Ken-ichi Kawarabayashi',
               'Michael Nett']
p29pdf_author_organizations=['Equipe LINKMEDIA','Campus Universitaire de Beaulieu','CNRS/IRISA Rennes','National Institute of Informatics',
                             'Equipe MISTIS INRIA','Google']
p29pdf_author_locations=['Campus Universitaire de Beaulieu','35042 Rennes Cedex, France','France','-1-2 Hitotsubashi, Chiyoda-ku Tokyo 101-8430, Japan',
                        'Japan','6-10-1 Roppongi, Minato-ku Tokyo 106-6126','Inovallée, 655, Montbonnot 38334 Saint-Ismier Cedex','Tokyo']

p29pdf_references_authors =['A. A. Balkema','L. de Haan','N. Bingham', 'C. Goldie','J. Teugels','N. Boujemaa', 'J. Fauqueur', 'M. Ferecatu', 'F. Fleuret',
                            'V. Gouet', 'B. LeSaux','H. Sahbi','C. Bouveyron', 'G. Celeux', 'S. Girard','J. Bruske', 'G. Sommer',
                           'F. Camastra','A. Vinciarelli','S. Coles','J. Costa' ,'A. Hero','T. de Vries', 'S. Chawla','M. E. Houle',
                           'R. A. Fisher','L. H. C. Tippett','M. I. Fraga Alves', 'L. de Haan','T. Lin','M. I. Fraga Alves', 'M. I. Gomes','L. de Haan',
                           'B. V. Gnedenko',' A. Gupta', 'R. Krauthgamer','J. R. Lee','A. Gupta', 'R. Krauthgamer','J. R. Lee','M. Hein','J.-Y. Audibert',
                           'B. M. Hill','M. E. Houle','M. E. Houle','M. E. Houle','M. E. Houle', 'H. Kashima', 'M. Nett','M. E. Houle', 'X. Ma', 'M. Nett',
                            'V. Oria','M. E. Houle', 'X. Ma', 'V. Oria','J. Sun','M. E. Houle','M. Nett','H. Jegou', 'R. Tavenard', 'M. Douze','L. Amsaleg',
                           'I. Jollie','D. R. Karger','M. Ruhl','J. Karhunen','J. Joutsensalo','Y. LeCun', 'L. Bottou', 'Y. Bengio', 'P. Haner',
                           'J. Pickands, III','C. R. Rao','S. T. Roweis','L. K. Saul','A. Rozza', 'G. Lombardi', 'C. Ceruti', 'E. Casiraghi', 'P. Campadelli',
                           'B. Scholkopf', 'A. J. Smola','K.-R. Muller','U. Shaft','R. Ramakrishnan',' F. Takens','J. Tenenbaum', 'V. D. Silva','J. Langford',
                           'J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. Venna','S. Kaski',
                           'P. Verveer','R. Duin','J. von Brunken', 'M. E. Houle', 'A. Zimek','J. von Brunken', 'M. E. Houle','A. Zimek']

p29pdf_allauthors=['Laurent Amsaleg','Stéphane Girard','Oussama Chelly','Teddy Furon','Michael E. Houle','Ken-ichi Kawarabayashi',
               'Michael Nett','A. A. Balkema','L. de Haan','N. Bingham', 'C. Goldie','J. Teugels','N. Boujemaa', 'J. Fauqueur', 'M. Ferecatu', 'F. Fleuret',
                            'V. Gouet', 'B. LeSaux','H. Sahbi','C. Bouveyron', 'G. Celeux', 'S. Girard','J. Bruske', 'G. Sommer',
                           'F. Camastra','A. Vinciarelli','S. Coles','J. Costa' ,'A. Hero','T. de Vries', 'S. Chawla','M. E. Houle',
                           'R. A. Fisher','L. H. C. Tippett','M. I. Fraga Alves', 'L. de Haan','T. Lin','M. I. Fraga Alves', 'M. I. Gomes','L. de Haan',
                           'B. V. Gnedenko',' A. Gupta', 'R. Krauthgamer','J. R. Lee','A. Gupta', 'R. Krauthgamer','J. R. Lee','M. Hein','J.-Y. Audibert',
                           'B. M. Hill','M. E. Houle','M. E. Houle','M. E. Houle','M. E. Houle', 'H. Kashima', 'M. Nett','M. E. Houle', 'X. Ma', 'M. Nett',
                            'V. Oria','M. E. Houle', 'X. Ma', 'V. Oria','J. Sun','M. E. Houle','M. Nett','H. Jegou', 'R. Tavenard', 'M. Douze','L. Amsaleg',
                           'I. Jollie','D. R. Karger','M. Ruhl','J. Karhunen','J. Joutsensalo','Y. LeCun', 'L. Bottou', 'Y. Bengio', 'P. Haner',
                           'J. Pickands, III','C. R. Rao','S. T. Roweis','L. K. Saul','A. Rozza', 'G. Lombardi', 'C. Ceruti', 'E. Casiraghi', 'P. Campadelli',
                           'B. Scholkopf', 'A. J. Smola','K.-R. Muller','U. Shaft','R. Ramakrishnan',' F. Takens','J. Tenenbaum', 'V. D. Silva','J. Langford',
                           'J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. Venna','S. Kaski',
                           'P. Verveer','R. Duin','J. von Brunken', 'M. E. Houle', 'A. Zimek','J. von Brunken', 'M. E. Houle','A. Zimek']


print "There are %r authors" % len(p29pdf_authors)
print  # white space
print "There are %r author organizations" %len(p29pdf_author_organizations)
print 
print "There are %r author locations" % len(p29pdf_author_locations)
print  
print "There are %r authors in the references" %len(p29pdf_references_authors)

There are 7 authors

There are 6 author organizations

There are 8 author locations

There are 106 authors in the references


In [224]:
len(set(compare) & set(p29pdf_allauthors))

0

In [225]:
len(p29pdf_allauthors)

113

In [226]:
re.sub('[\s]'," ",document)[9300:10500]
#regexp.search(re.sub('[\s]'," ",document)).group(1)[4900:6000]

''

In [227]:
extraction(document)

{'locations': ['Microsoft Bellevue',
  'WA',
  'USA',
  'Red Prairie',
  'Israel',
  'England',
  'Sydney',
  'NSW',
  'Australia'],
 'organizations': ['Microsoft',
  'Microsoft',
  'Microsoft',
  'Business Intelligence',
  'Blue Martini Software',
  'Blue',
  'Silicon Graphics',
  'Graphics',
  'Stanford University',
  'Blue Martini Software',
  'Technion',
  'National Academy of Engineering',
  'Computer Science',
  'Google',
  'Yahoo',
  'eBay',
  'Netflix',
  'Facebook',
  'ACM'],
 'persons': ['Ron Kohavi',
  'Ronny Kohavi',
  'Blue Martini',
  'Kohavi',
  'Kohavi',
  'Jim Gray',
  'Carla Brodley',
  'Microsofts Bing',
  'Sir Ronald A',
  'KDD15']}

In [228]:
class entities(object):
  def __init__(self):
    self.persons = extraction(document)['persons']
    self.organizations = extraction(document)['organizations']

my_shape = entities()
my_shape.persons

['Ron Kohavi',
 'Ronny Kohavi',
 'Blue Martini',
 'Kohavi',
 'Kohavi',
 'Jim Gray',
 'Carla Brodley',
 'Microsofts Bing',
 'Sir Ronald A',
 'KDD15']

# Code to extract emails

In [229]:
tuple(get_emails(document))

('ohavi@outlook.com',)

# Code to get only the Title and Author Names

In [230]:
p=re.compile('(.*)(?=ABSTRACT)')
try:
    topsec = p.search(re.sub('[\s]'," ",document)).group(1)
except AttributeError:
    p=re.compile('(.*)(?=abstract)')
    topsec = p.search(re.sub('[\s]'," ",lower)).group(1)
topsec


'online controlled experiments:   lessons from running a/b/n tests for 12 years      ron kohavi   distinguished engineer & general manager,    analysis & experimentation   microsoft   bellevue, wa, usa   ronkohavi@outlook.com    in   he   2005     bio  ronny  kohavi is  a  microsoft  distinguished  engineer  and  for  the  general  manager  and  microsofts  analysis  at  experimentation  team  microsoft.  joined  microsoft  and  founded  the  experimentation  platform  team  in  2006.  he  was previously the director of  data  mining  and  personalization  at  amazon.com,  and  the  vice  president  of  business  intelligence  at  blue  martini  software,  which  went  public  in  2000,  and  later  acquired  by  red  prairie.  prior to joining blue martini, kohavi managed mineset project,  silicon  graphics  award-winning  product  for  data  mining  and  visualization. he joined silicon graphics after getting a ph.d. in  machine  learning  from  stanford  university,  where  he  led 

# Code to get Title Only (or most of it)

In [231]:
p=re.compile('(.+)(\\n\\n)')
q=re.compile('(?<=\\n\\n)(.+?)(?=\\n\\n)')
#p.search(document).group(1)+" "+q.search(document).group(1)
p.search(document).group(1)

'Online Controlled Experiments: '

# Code to get Abstract only

In [232]:
p= re.compile('(?<=ABSTRACT)(.+)(?=Categories and Subject Descriptors)')
try:
    p.search(re.sub('[\s]'," ",document)).group(1)
except AttributeError:
    p=re.compile('(?<=abstract)(.+)(?=categories and subject descriptors)')
    abstract=p.search(re.sub('[\s]'," ",lower)).group(1)
else:
    pass
abstract

'  the  internet  provides  developers  of  connected  software,  including web sites, applications, and devices, an unprecedented  opportunity to accelerate innovation by evaluating ideas quickly  and  accurately  using  trustworthy  controlled  experiments  (e.g.,  a/b  tests  and  their  generalizations).  from  front-end  user- interface  changes  to  backend  recommendation  systems  and  relevance  algorithms,  from  search  engines  (e.g.,  google,  microsofts  bing,  yahoo)  to  retailers  (e.g.,  amazon,  ebay,  netflix,  etsy)  to  social  networking  services  (e.g.,  facebook,  linkedin,  twitter)  to  travel  services  (e.g.,  expedia,  airbnb,  booking.com)  to  many  startups,  online  controlled  experiments  are now utilized to make data-driven decisions at a wide range of  companies.  while  the  theory  of  a  controlled  experiment  is  simple, and dates back to sir ronald a. fishers experiments at  the rothamsted agricultural experimental station in england in  the

# Code to get keywords only

In [233]:
p= re.compile('(?<=Keywords)(.+)(?=INTRODUCTION)')
try:
    keywords=p.search(re.sub('[\s]'," ",document)).group(1)
    try:
        p=re.compile('(?<=Keywords)(.+)')
        keywords=p.search(re.sub('[\s]'," ",document)).group(1)
    
    except AttributeError:
        print "Unexpected error:", sys.exc_info()[0]
except:
    print "This file does not have a keywords section or the section headers are different"
    pass
keywords;

This file does not have a keywords section or the section headers are different


In [234]:
re.findall('(?=keywords)(.+)',lower)

['keywords ']

In [235]:
p= re.compile('(?<=ABSTRACT)(.+)(?=Categories and Subject Descriptors)')
try:
    abstract = p.search(re.sub('[\s]'," ",document)).group(1)
except AttributeError:
    p=re.compile('(?<=abstract)(.+)(?=categories and subject descriptors)')
    abstract = p.search(re.sub('[\s]'," ",lower)).group(1)
else:
    pass
abstract

'  the  internet  provides  developers  of  connected  software,  including web sites, applications, and devices, an unprecedented  opportunity to accelerate innovation by evaluating ideas quickly  and  accurately  using  trustworthy  controlled  experiments  (e.g.,  a/b  tests  and  their  generalizations).  from  front-end  user- interface  changes  to  backend  recommendation  systems  and  relevance  algorithms,  from  search  engines  (e.g.,  google,  microsofts  bing,  yahoo)  to  retailers  (e.g.,  amazon,  ebay,  netflix,  etsy)  to  social  networking  services  (e.g.,  facebook,  linkedin,  twitter)  to  travel  services  (e.g.,  expedia,  airbnb,  booking.com)  to  many  startups,  online  controlled  experiments  are now utilized to make data-driven decisions at a wide range of  companies.  while  the  theory  of  a  controlled  experiment  is  simple, and dates back to sir ronald a. fishers experiments at  the rothamsted agricultural experimental station in england in  the

# Code to get Categories and Subject Descriptors

In [236]:
p= re.compile('(?<=Categories and Subject Descriptors)(.+)(?=Keywords)')
try:
    catnsub = re.findall('\[(.*?)\]',p.search(re.sub('[\s]'," ",document)).group(1))
    try:
        catnsub = p.search(re.sub('[\s]'," ",document)).group(1)
    except AttributeError:
        print "Unexpected error:", sys.exc_info()[0]
except:
    print "This file does not have a categories and subject descriptors section or the section headers are different"
    pass
catnsub

'  G.3 Probability and Statistics/Experimental Design: controlled  experiments; randomized experiments; A/B testing.  General Terms  Measurement; Design; Experimentation    '

# Code to get Body only

In [237]:
p= re.compile('(?<=INTRODUCTION)(.+)(?=CONCLUSION)')
try:
    body = p.search(re.sub('[\s]'," ",document)).group(1)
    try:
        p= re.compile('(?<=introduction)(.+)(?=conclusion)')
        body = p.search(re.sub('[\s]'," ",lower)).group(1)
    except AttributeError:
        print "Unexpected error:", sys.exc_info()[0]
except:
    print "This file does not have a body or the section headers are different"
    pass
body;

This file does not have a body or the section headers are different


# Code to get CONCLUSION only

In [238]:
p= re.compile('(?<=CONCLUSION)(.+)(?=REFERENCES)')
try:
    conclusion = p.search(re.sub('[\s]'," ",document)).group(1)
    try:
        p= re.compile('(?<=conclusion)(.+)(?=references)')
        conclusion = p.search(re.sub('[\s]'," ",lower)).group(1)
    except AttributeError:
        print "Unexpected error:", sys.exc_info()[0]
except AttributeError:
    print "This may not have a conclusion or has different section headers"
    pass
conclusion

This may not have a conclusion or has different section headers


'S  We presented a system called TIMEMACHINE for automatic time- line generation for entities in a knowledge base. The timeline gen- eration problem is formulated in a submodular optimization frame- work that jointly optimizes for relevance, content diversity and tem- poral diversity. Web-based co-occurrence signals are used to de- termine the relevance of other entities and dates to the timeline subject. We proved that an efcient greedy approximation algo- rithm achieves near-optimal performance. The proposed approach is evaluated through a comprehensive series of user studies demon- strating that both temporal diversity and content diversity are cru- cial, and that web-based co-occurrence signals signicantly im- prove over a baseline model that relies on global importance.  Acknowledgments. We thank Evgeniy Gabrilovich for many help- ful discussions, Arun Chaganty, Stefanie Jegelka, Karthik Raman, Sujith Ravi, and Ravi Kumar for their insights on submodular opti- mization, Jeff Tamer

# Code to get References only

In [239]:
p= re.compile('(?<=REFERENCES)(.+)')
try:
    references = p.search(re.sub('[\s]'," ",document)).group(1)
    try:
        p= re.compile('(?<=references)(.+)')
        references = p.search(re.sub('[\s]'," ",lower)).group(1)
    except AttributeError:
        print "Unexpected error:", sys.exc_info()[0]
except AttributeError:
    print "This file may not have references or has different section headers"
    pass
references  

This file may not have references or has different section headers


' [1] A. Ahmed, C. H. Teo, S. Vishwanathan, and A. Smola. Fair and  balanced: Learning to present news stories. In WSDM, 2012.  [2] J. Allan, R. Gupta, and V. Khandelwal. Temporal summaries of new  topics. In SIGIR, 2001.  [15] D. Graus, M.-H. Peetz, D. Odijk, O. de Rooij, and M. de Rijke.  yourHistorySemantic linking for a personalized timeline of historic events. Workshop: LinkedUp Challenge at OKCon, 2013.  [16] T. Huet, J. Biega, and F. M. Suchanek. Mining history with le monde.  In AKBC, 2013.  [17] H. Ji, T. Cassidy, Q. Li, and S. Tamang. Tackling representation,  annotation and classication challenges for temporal knowledge base population. KAIS, 2013.  [18] A. Kannan, S. Baker, K. Ramnath, J. Fiss, D. Lin, L. Vanderwende,  R. Ansary, A. Kapoor, Q. Ke, M. Uyttendaele, et al. Mining text snippets for images on the web. In SIGKDD, 2014.  [19] S. M. Katz. Estimation of probabilities from sparse data for the  language model component of a speech recognizer. In IEEE Trans Sig. Proces

# Code to count the number of references

In [240]:
'''
create a match of all references by counting the number of integers enclosed by brackets (i.e. '[2]').  

This is how references are labeled in the research papers
'''
# test to see if references exist
if references in globals():
    len(re.findall('\[(.*?)\]',regexp.search(re.sub('[\s]'," ",document)).group(1)))
    
else:
    print "This file does not have references or the section headers are different"

This file does not have references or the section headers are different


# NLTK standard chunk comparison to hand labeled entities

In [241]:
"returns named entity chunks in a given text"
testtagged = nltk.pos_tag(nltk.word_tokenize(abstract))
testentities = nltk.chunk.ne_chunk(testtagged)

machinelist_persons=[]
machinelist_orgs=[]

for l in testentities:
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'PERSON':
            if " ".join(map(itemgetter(0), l)) not in machinelist_persons:
                machinelist_persons.append(" ".join(map(itemgetter(0), l)))

    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'ORGANIZATION':
            if " ".join(map(itemgetter(0), l)) not in machinelist_orgs:
                machinelist_orgs.append(" ".join(map(itemgetter(0), l)))
print len(machinelist_persons),len(p19pdf_authors)                
set(machinelist_persons) & set(p19pdf_authors)

0 6


set()

In [243]:
'''
Using explanations from this page to estimate precision and recall
http://www.kdnuggets.com/faq/precision-recall.html
'''
inboth = float(len(set(machinelist_persons) & set(p19pdf_authors)))
ext_len=float(len(machinelist_persons))
true_len=float(len(p19pdf_authors))

d = {'Predicted Negative': [0,0], 'Predicted Positive': [ext_len-true_len,inboth]}
metrics = pd.DataFrame(d, index=['Negative Cases','Positive Cases'])
try:
    print "The \"Accuracy\" is %r.\nThe \"Recall\" is %r.\nThe \"Precision\" is %r." % ((round((inboth/ext_len)*100)),(round(inboth/true_len)*100),(inboth/(inboth+ext_len-true_len))*100)
except ZeroDivisionError:
    print "Not enough data"
metrics

Not enough data


,Predicted Negative,Predicted Positive
Negative Cases,0,-6
Positive Cases,0,0


In [245]:
"returns named entity chunks in a given text"
testtagged = nltk.pos_tag(nltk.word_tokenize(references))
testentities = nltk.chunk.ne_chunk(testtagged)

machinelist_persons=[]
machinelist_orgs=[]

for l in testentities:
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'PERSON':
            if " ".join(map(itemgetter(0), l)) not in machinelist_persons:
                machinelist_persons.append(" ".join(map(itemgetter(0), l)))

    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'ORGANIZATION':
            if " ".join(map(itemgetter(0), l)) not in machinelist_orgs:
                machinelist_orgs.append(" ".join(map(itemgetter(0), l)))
print len(machinelist_persons),len(p19pdf_references_authors)                
set(machinelist_persons) & set(p19pdf_references_authors)

40 152


{'J. Biega',
 'J. Fiss',
 'J. Jacobs',
 'J. Magnani',
 'J. Novak',
 'J. VanBriesen',
 'J. Wang',
 'J. Yang'}

In [265]:
# split the list of hand labeled names to just keep the last name for comparison

newlist=[]
#print machinelist_persons
#print p19pdf_references_authors
for l in p19pdf_references_authors:
    for m in l.split():
        if len(m)<=2:
            pass
        else:
            newlist.extend([m])        
sorted(newlist)

['Ahmed',
 'Alai',
 'Allan',
 'Allan',
 'Althoff',
 'Ansary',
 'Baeza-Yates',
 'Baker',
 'Bamman',
 'Benczr',
 'Bennett',
 'Biega',
 'BilmesX.',
 'Bollacker',
 'C.-Y.',
 'Calinescu',
 'Carbonell',
 'Cardie',
 'Cardie',
 'Carterette',
 'Cassidy',
 'Ceroni',
 'Chekuri',
 'Chickering',
 'Dang',
 'Dasgupta',
 'Dong',
 'Dong',
 'Dubinko',
 'Dumais',
 'Elbassuoni',
 'Evans',
 'Faloutsos',
 'Feige',
 'Fisher',
 'Fisichella',
 'Fiss',
 'Gabrilovich',
 'Georgescu',
 'Glance',
 'Goldstein',
 'Golovin',
 'Graus',
 'Guestrin',
 'Guestrin',
 'Guo',
 'Gupta',
 'Han',
 'Heitz',
 'Horn',
 'Horvitz',
 'Huet',
 'Jacobs',
 'Joachims',
 'Jones',
 'Kannan',
 'Kapoor',
 'Kasneci',
 'Katz',
 'Khandelwal',
 'Kirkpatrick',
 'Krause',
 'Krause',
 'Kumar',
 'Kumar',
 'Lao',
 'Leskovec',
 'Leskovec',
 'Lin',
 'Lin',
 'Lin',
 'Ling',
 'M.-H.',
 'Magnani',
 'Mazeika',
 'Minoux',
 'Murphy',
 'Murphy',
 'Naini',
 'Nemhauser',
 'Novak',
 'Ntarmos',
 'Odijk',
 'Paritosh',
 'Peetz',
 'Preda',
 'Preda',
 'Qian',
 'Raghav

In [247]:
'''
Using explanations from this page to estimate precision and recall
http://www.kdnuggets.com/faq/precision-recall.html
'''
inboth = float(len(set(machinelist_persons) & set(p19pdf_references_authors)))
ext_len=float(len(machinelist_persons))
true_len=float(len(p19pdf_references_authors))

d = {'Predicted Negative': [0,0], 'Predicted Positive': [abs(ext_len-true_len),inboth]}
metrics = pd.DataFrame(d, index=['Negative Cases','Positive Cases'])
print "The \"Accuracy\" is %r.\nThe \"Recall\" is %r.\nThe \"Precision\" is %r." % ((round((inboth/ext_len)*100)),(round(inboth/true_len)*100),(inboth/(inboth+ext_len-true_len))*100)
#,,
metrics

The "Accuracy" is 20.0.
The "Recall" is 0.0.
The "Precision" is -7.6923076923076925.


,Predicted Negative,Predicted Positive
Negative Cases,0,112
Positive Cases,0,8
